# LangChain Expression Language (LCEL)

In [2]:
#!pip install pydantic==1.10.8

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOllama
from langchain.schema.output_parser import StrOutputParser

## Simple Chain

In [11]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOllama(model='gemma:2b-instruct')
output_parser = StrOutputParser()

In [12]:
chain = prompt | model | output_parser

In [13]:
chain.invoke({"topic": "bears"})

"What do you call a bear that's always hungry?\n\nA bear-ing bear!"

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [25]:
# from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [34]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma(collection_name="my_collection", embedding_function=embedding_function)

texts = ["harrison worked at kensho", "bears like to eat honey"]

db.add_texts(texts)
retriever = db.as_retriever()

In [35]:
retriever.get_relevant_documents("where did harrison work?",)

[Document(page_content='harrison worked at kensho'),
 Document(page_content='harrison worked at kensho'),
 Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [36]:
retriever.get_relevant_documents("what do bears like to eat")

[Document(page_content='bears like to eat honey'),
 Document(page_content='bears like to eat honey'),
 Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

In [37]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [38]:
from langchain.schema.runnable import RunnableMap

In [39]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [40]:
chain.invoke({"question": "where did harrison work?"})

'The context does not specify where harrison worked, so I cannot answer this question from the context.'

In [17]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [18]:
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

## Fallbacks

In [48]:
from langchain.llms import OpenAI
import json

**Note**: Due to the deprecation of OpenAI's model `text-davinci-001` on 4 January 2024, you'll be using OpenAI's recommended replacement model `gpt-3.5-turbo-instruct` instead.

In [49]:
simple_model = ChatOllama(model="gemma:2b-instruct")

simple_chain = simple_model | json.loads

In [50]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [51]:
simple_model.invoke(challenge)

AIMessage(content='```json\n{\n  "title": "The Song of the Birds",\n  "author": "John Lennon",\n  "first_line": "The birds all sing a melody."\n}\n\n{\n  "title": "Hamlet",\n  "author": "William Shakespeare",\n  "first_line": "To be or not to be, that is the question."\n}\n\n{\n  "title": "The Lord of the Rings",\n  "author": "J.R.R. Tolkien",\n  "first_line": "The Lord of the Rings, a creature of fire and ice."\n}\n```', response_metadata={'model': 'gemma:2b-instruct', 'created_at': '2024-03-23T12:01:44.670433Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 5841175200, 'prompt_eval_count': 29, 'prompt_eval_duration': 885432000, 'eval_count': 137, 'eval_duration': 4897934000})

**Note**: The next line is expected to fail.

In [ ]:
try:
    simple_chain.invoke(challenge)
finally:
    print("yep")

In [53]:
final_chain = simple_chain.with_fallbacks([chain])

final_chain.invoke(challenge)

## Interface

In [43]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOllama(model="gemma:2b-instruct")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [44]:
chain.invoke({"topic": "bears"})

"What do you call a bear that's always hungry?\n\nA bear-licious bear!"

In [45]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

["What do you call a bear that's always hungry?\n\nA bear-kfast!",
 "What do you call a frog that's always complaining?\n\nA frogger!"]

In [46]:
for t in chain.stream({"topic": "bears"}):
    print(t)

What
 do
 you
 call
 a
 bear
 that
'
s
 always
 hungry
?



A
 bear
-
licious
 bear
!



In [47]:
response = await chain.ainvoke({"topic": "bears"})
response

"What do you call a bear that's too heavy?\n\nA grizzly load!"

## Binding functions

In [55]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [58]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOllama(model="gemma:2b-instruct",temperature=0).bind(functions=functions)

In [60]:
runnable = prompt | model

# runnable.invoke({"input": "what is the weather in sf"})